### Importar bibliotecas

Pandas: tratar dados de forma tabular semelhante ao Excel (as pd é apelido)

Collections: biblioteca para trabalhar com dicionáros (dicts) de forma ordenada

Numpy: biblioteca para operações matemáticas

In [1]:
import pandas as pd
import collections
import numpy as np

### Definir função objetivo

In [2]:
z = collections.OrderedDict({'x1':10,'x2':12})
lista_variaveis = z.keys()
print(lista_variaveis)
print(z)

odict_keys(['x1', 'x2'])
OrderedDict([('x1', 10), ('x2', 12)])


### Definir lista de restrições

In [147]:
restricoes = [collections.OrderedDict({'x1':1,'x2':1,'op':'<=','b':100}),
        collections.OrderedDict({'x1':1,'x2':3,'op':'<=','b':270})]
    
print(restricoes)

[OrderedDict([('x1', 1), ('x2', 1), ('op', '<='), ('b', 100)]), OrderedDict([('x1', 1), ('x2', 3), ('op', '<='), ('b', 270)])]


### Cria a tabela inserindo a linha objetivo

In [148]:
# Valor de Z na função objetivo sempre é 1
linha_obj = collections.OrderedDict({'z': [1]})

for chave, valor in z.items():
    linha_obj[chave] = [valor*(-1)]

linha_obj['b'] = [0]
linha_obj = pd.DataFrame.from_dict(linha_obj)
linha_obj

,z,x1,x2,b
0,1,-10,-12,0


### Inserir as restrições na tabela

In [149]:
# Cria a linha de cada restrição recebendo o dicionário da restrição e quantidade de folgas
def construir_restricao(restricao, qtd_folgas) : 
    linha = collections.OrderedDict()
    # Valor de Z nas restrições sempre é zero
    linha['z'] = [0]
    for chave, valor in restricao.items():
        if chave != 'b':
            linha[chave] = valor

    if (linha['op'] == '<=') or (linha['op'] == '>='):
        linha['xF' + str(qtd_folgas + 1)] = 1
        qtd_folgas += 1
        
    del(linha['op'])
    linha['b'] = restricao['b']
    
    return linha, qtd_folgas

In [150]:
# Inserindo cada restrições, da lista restrições, na tabela criada anteriormente
qtd_folgas = 0
# Tratamento para 'resetar' a tabela em tempo de execução
tabela = linha_obj.copy()
for restricao in restricoes:
    linha, qtd_folgas = construir_restricao(restricao, qtd_folgas)
    linha = pd.DataFrame.from_dict(linha)
    tabela = tabela.append(linha, ignore_index=True)
tabela = tabela.fillna(0)

In [151]:
tabela

,b,x1,x2,xF1,xF2,z
0,0,-10,-12,0.0,0.0,1
1,100,1,1,1.0,0.0,0
2,270,1,3,0.0,1.0,0


### Função responsável por identificar a coluna que entra na função objetivo

In [152]:
def identificar_coluna_entra(tabela):
    primeira_linha = tabela.iloc[0]
    lista_valores = []
    
    for chave, valor in primeira_linha.items():
        if chave in lista_variaveis:
            lista_valores.append(valor)
            
    menor_valor = np.min(lista_valores)
    
    for chave, valor in primeira_linha.items():
        if chave in lista_variaveis:
            if menor_valor == valor:
                menor_chave = chave
                break
    
    return menor_chave
        

### Função responsável por identificar a linha que sai

In [153]:
def identificar_linha_sai(tabela, coluna_entra):
    # Remover a primeira linha (função objetivo)
    tabela = tabela.drop([0])
    # Cria nova coluna do resultado da divisão entre 'b' e a coluna que entra (parâmetro)
    tabela['b/coluna_entra'] = tabela['b'] / tabela[coluna_entra]
    # Encontra o menor valor da coluna de divisão
    menor_divisao = np.min(tabela['b/coluna_entra'])
    # Identifica o primeiro índice da linha que possui o valor igual ao da menor divisão
    indice = list(tabela.loc[tabela['b/coluna_entra'] == menor_divisao].index)[0]
    
    return indice

In [154]:
coluna_entra = identificar_coluna_entra(tabela)

linha_sai = identificar_linha_sai(tabela, coluna_entra)

# Encontra o elemento pivô da interseção entre coluna que entra e linha que sai
elemento_pivo = tabela[coluna_entra][linha_sai]

In [155]:
def calcular_nova_linha_pivo(tabela, linha_sai, elemento_pivo) :
    tabela_temp = tabela.copy()
    tabela_temp.iloc[linha_sai] = tabela_temp.iloc[linha_sai]/elemento_pivo
    return tabela_temp.iloc[linha_sai]

In [156]:
nova_linha_pivo = calcular_nova_linha_pivo(tabela, linha_sai, elemento_pivo)
print(nova_linha_pivo)

b      90.000000
x1      0.333333
x2      1.000000
xF1     0.000000
xF2     0.333333
z       0.000000
Name: 2, dtype: float64


### Função responsável por calcular a nova tabela a partir da tabela original e nova linha pivô

In [263]:
def calcular_nova_tabela(tabela, nova_linha_pivo, coluna_entra, linha_sai):
    # Remove a primeira linha da tabela original (função objetivo)
    temp_tabela_sem_pivo = tabela.drop([linha_sai])
    # Copia a tabela existente para uma nova e remove todos seus dados
    nova_tabela = tabela.copy()
    nova_tabela.drop(nova_tabela.index, inplace=True)
    
    ## PODE DAR PROBLEMA!!!!
    for i in (0, len(temp_tabela_sem_pivo) -1):
        # Calcula nova linha
        nova_linha = nova_linha_pivo * (temp_tabela_sem_pivo[coluna_entra][i] * -1)
        # Adiciona com a linha correspondente do indice da tabela original
        nova_linha = nova_linha + temp_tabela_sem_pivo.iloc[i]
        # Adiciona a nova linha calculada na nova tabela
        nova_tabela = nova_tabela.append(nova_linha, ignore_index=True)
    
    # Adiciona a linha pivô na nova tabela
    nova_tabela = nova_tabela.append(nova_linha_pivo)
    
    return nova_tabela

In [272]:
nova_tabela = calcular_nova_tabela(tabela, nova_linha_pivo, coluna_entra, linha_sai)

### Função responsável por analisar a primeira linha da nova tabela (função objetivo) e verificar se o resultado obtido é ótimo

In [286]:
def is_resultado_otimo(nova_tabela):
    menor_valor = min(nova_tabela.iloc[0])
    if menor_valor < 0:
        return False
    return True